In [2]:
# später anpassen für colab und lokal
# move directory to /realwaste
import os
os.chdir('/realwaste')

data_path = "./data"

In [3]:
from pytorch_lightning import LightningDataModule
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, random_split
from torchvision import transforms
import os

class ConfigurableDataModule(LightningDataModule):
    """Class wraper für mit austauschbaren transforms"""
    def __init__(self, data_dir: str, batch_size: int, transform):
        super().__init__()
        self.data_dir = data_dir
        self.batch_size = batch_size
        self.transform = transform

    def setup(self, stage=None):
        # Erstellen des Datensatzes als Instanz von ImageFolder
        full_dataset = ImageFolder(root=self.data_dir, transform=self.transform)
        # Setzen der Trainingsset/Validierungsset Größe
        train_size = int(0.8 * len(full_dataset))
        val_size = len(full_dataset) - train_size
        # Zufälliges aufteilen in Training- und Validierungdatensatz
        self.train_dataset, self.val_dataset = random_split(full_dataset, [train_size, val_size])

    def train_dataloader(self):
        # Setzen des Traindataloader
        return DataLoader(self.train_dataset, batch_size=self.batch_size, shuffle=True, num_workers=os.cpu_count())

    def val_dataloader(self):
        # Setzen des Validation Dataloader
        return DataLoader(self.val_dataset, batch_size=self.batch_size, num_workers=os.cpu_count())

/usr/local/lib/python3.10/dist-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/usr/local/lib/python3.10/dist-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [4]:
from PIL import Image, ImageDraw, ImageFont
import pytorch_lightning as pl
import torch
import torchmetrics
import torchvision
import torchvision.utils as vutils
from torchvision.transforms.functional import to_pil_image, to_tensor
import random

class BaseWasteClassifier(pl.LightningModule):
    
    CLASS_NAMES =  ['Cardboard', 'Food Organics', 'Glass', 'Metal', 'Miscellaneous Trash', 'Paper', 'Plastic', 'Textile Trash', 'Vegetation']

    def __init__(self, num_classes: int):
        super().__init__()
        self.num_classes = num_classes
        # Placeholder for the actual model, to be defined in subclasses
        self.model = None

        # Initialize metrics common to all models
        self.accuracy = torchmetrics.Accuracy(task='multiclass', num_classes=num_classes, average='macro')
        self.precision = torchmetrics.Precision(task='multiclass', num_classes=num_classes, average='weighted')
        self.recall = torchmetrics.Recall(task='multiclass', num_classes=num_classes, average='weighted')

    def forward(self, x):
        # Subclasses should implement this method
        raise NotImplementedError("This method should be overridden by subclasses.")

    def training_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = torch.nn.functional.cross_entropy(logits, y)
        acc = self.accuracy(torch.argmax(logits, dim=1), y)
        self.log('train_loss', loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        self.log('train_acc', acc, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = torch.nn.functional.cross_entropy(logits, y)
        acc = self.accuracy(torch.argmax(logits, dim=1), y)
        predictions = torch.argmax(logits, dim=1)  # Convert logits to predicted class indices

        self.log('val_loss', loss, prog_bar=True)
        self.log('val_acc', acc, prog_bar=True)

        if random.random() < 0.1:  # Log images randomly
            self.log_images_with_labels(x, y, predictions, batch_idx)  # Pass predictions correctly


    def log_images_with_labels(self, images, labels, predictions, batch_idx):
        """Log a batch of images with their actual and predicted labels to TensorBoard."""
        annotated_images = []

        for i in range(images.size(0)):
            img = images[i]
            actual_label = labels[i].item()
            predicted_label = predictions[i].item()

            # Unnormalize the image for visualization
            img = self.unnormalize(img)  # Make sure to call with self if it's an instance method

            # Convert to PIL Image for easy manipulation
            pil_img = to_pil_image(img)
            draw = ImageDraw.Draw(pil_img)
            annotation_text = f'Actual: {self.CLASS_NAMES[actual_label]},\n Predicted: {self.CLASS_NAMES[predicted_label]}'
            draw.text((10, 10), annotation_text, fill="white")

            # Convert back to tensor and add to list
            annotated_img = to_tensor(pil_img)
            annotated_images.append(annotated_img.unsqueeze(0))  # Add batch dimension

        # Stack all annotated images into a single tensor for logging
        annotated_images_tensor = torch.cat(annotated_images, dim=0)
        img_grid = torchvision.utils.make_grid(annotated_images_tensor, nrow=4)

        # Log the grid of annotated images
        self.logger.experiment.add_image(f'Validation Images, Batch {batch_idx}', img_grid, self.current_epoch)


    def unnormalize(self, image, mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]):
        """Revert normalization of an image tensor."""
        image = image.clone()  # Clone the tensor to avoid in-place operations
        for t, m, s in zip(image, mean, std):
            t.mul_(s).add_(m)  # Multiply by std and add mean
        return image

    def configure_optimizers(self):
        # Subclasses can override this if needed
        optimizer = torch.optim.SGD(self.model.parameters(), lr=0.001, momentum=0.9)
        return optimizer


In [24]:
import torch.nn as nn

class SimpleCNN(BaseWasteClassifier):
    def __init__(self, num_classes=9, optimizer_name='Adam', lr=1e-3):
        super().__init__(num_classes)
        self.optimizer_name = optimizer_name
        self.lr = lr
        self.model = nn.Sequential(
            nn.Conv2d(3, 16, 3, padding=1),  # Beispielarchitektur
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(16, 32, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(32, 64, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Flatten(),
            nn.Linear(64 * 28 * 28, 512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, num_classes)
        )

    def forward(self, x):
        return self.model(x)

    def configure_optimizers(self):
        if self.optimizer_name == 'Adam':
            optimizer = torch.optim.Adam(self.parameters(), lr=self.lr)
        elif self.optimizer_name == 'SGD':
            optimizer = torch.optim.SGD(self.parameters(), lr=self.lr, momentum=0.9)
        elif self.optimizer_name == 'RMSprop':
            optimizer = torch.optim.RMSprop(self.parameters(), lr=self.lr)
        else:
            raise ValueError(f"Unbekannter Optimierer: {self.optimizer_name}")

        return optimizer

In [25]:
import optuna
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import EarlyStopping
from torchvision import transforms

data_module = ConfigurableDataModule(data_dir=data_path, batch_size=32, transform=None)

def objective(trial):
    # Hyperparameter für Transformationen
    resize = trial.suggest_categorical('resize', [224, 256])
    crop = trial.suggest_categorical('crop', [192, 224])
    rotate = trial.suggest_uniform('rotate', 0, 25)  # Grad der Rotation
    horizontal_flip_prob = trial.suggest_uniform('horizontal_flip_prob', 0, 1)

    # Transformationen definieren
    transform = transforms.Compose([
        transforms.Resize(resize),
        transforms.RandomCrop(crop),
        transforms.RandomRotation(rotate),
        transforms.RandomHorizontalFlip(p=horizontal_flip_prob),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    
    batch_size = trial.suggest_categorical('batch_size', [16, 32, 64, 128])

    # Datenmodul mit den neuen Transformationen aktualisieren
    data_module.transform = transform
    data_module.batch_size = batch_size
    data_module.setup()  # Neuinitialisierung des Datenmoduls mit den aktualisierten Transformationen
    
    # Definition der Hyperparameterbereiche
    lr = trial.suggest_loguniform('lr', 1e-5, 1e-1)
    
    # Modell- und Datenmodul-Konfiguration
    model = SimpleCNN(num_classes=9, optimizer_name='Adam', lr=1e-3)

    # Setze den Optimierer im Modell
    model.configure_optimizers = optimizer

    # Trainer konfigurieren
    trainer = Trainer(  
        max_epochs=10,
        logger=False  # Optional: Deaktiviere Logging für Optuna-Optimierung
    )

    # Führe das Training durch
    trainer.fit(model, datamodule=data_module)

    # Gebe die beste Validierungs-Genauigkeit des Trials zurück
    val_accuracy = trainer.callback_metrics["val_acc"].item()
    return val_accuracy

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=1)  # Anzahl der Trials

print("Beste Hyperparameter: ", study.best_trial.params)


[I 2024-02-22 22:49:07,586] A new study created in memory with name: no-name-7104bc06-72c0-4ffd-bb62-cfacf47660a7
/tmp/ipykernel_1061/2502879651.py:12: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  rotate = trial.suggest_uniform('rotate', 0, 25)  # Grad der Rotation
/tmp/ipykernel_1061/2502879651.py:13: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  horizontal_flip_prob = trial.suggest_uniform('horizontal_flip_prob', 0, 1)
/tmp/ipykernel_1061/2502879651.py:33: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  lr = trial.suggest_loguniform('lr', 1e-5, 1

AttributeError: 'SGD' object has no attribute '__code__'

In [20]:
Trainer?

Init signature:
Trainer(
    *,
    accelerator: Union[str, pytorch_lightning.accelerators.accelerator.Accelerator] = 'auto',
    strategy: Union[str, pytorch_lightning.strategies.strategy.Strategy] = 'auto',
    devices: Union[List[int], str, int] = 'auto',
    num_nodes: int = 1,
    precision: Union[Literal[64, 32, 16], Literal['transformer-engine', 'transformer-engine-float16', '16-true', '16-mixed', 'bf16-true', 'bf16-mixed', '32-true', '64-true'], Literal['64', '32', '16', 'bf16'], NoneType] = None,
    logger: Union[pytorch_lightning.loggers.logger.Logger, Iterable[pytorch_lightning.loggers.logger.Logger], bool, NoneType] = None,
    callbacks: Union[List[pytorch_lightning.callbacks.callback.Callback], pytorch_lightning.callbacks.callback.Callback, NoneType] = None,
    fast_dev_run: Union[int, bool] = False,
    max_epochs: Optional[int] = None,
    min_epochs: Optional[int] = None,
    max_steps: int = -1,
    min_steps: Optional[int] = None,
    max_time: Union[str, datetime.

In [18]:
Trainer?

Init signature:
Trainer(
    *,
    accelerator: Union[str, pytorch_lightning.accelerators.accelerator.Accelerator] = 'auto',
    strategy: Union[str, pytorch_lightning.strategies.strategy.Strategy] = 'auto',
    devices: Union[List[int], str, int] = 'auto',
    num_nodes: int = 1,
    precision: Union[Literal[64, 32, 16], Literal['transformer-engine', 'transformer-engine-float16', '16-true', '16-mixed', 'bf16-true', 'bf16-mixed', '32-true', '64-true'], Literal['64', '32', '16', 'bf16'], NoneType] = None,
    logger: Union[pytorch_lightning.loggers.logger.Logger, Iterable[pytorch_lightning.loggers.logger.Logger], bool, NoneType] = None,
    callbacks: Union[List[pytorch_lightning.callbacks.callback.Callback], pytorch_lightning.callbacks.callback.Callback, NoneType] = None,
    fast_dev_run: Union[int, bool] = False,
    max_epochs: Optional[int] = None,
    min_epochs: Optional[int] = None,
    max_steps: int = -1,
    min_steps: Optional[int] = None,
    max_time: Union[str, datetime.

In [5]:
from pytorch_lightning import Trainer
from pytorch_lightning.loggers import TensorBoardLogger

def train_model(model, data_module, log_dir="tb_logs", max_epochs=5, logger_name="model_logs"):
    # Starten das Trainingsprozesses
    logger = TensorBoardLogger(log_dir, name=logger_name)
    trainer = Trainer(max_epochs=max_epochs, logger=logger)
    trainer.fit(model, datamodule=data_module)